In [3]:
CREATE DATABASE lab2;


Commands completed successfully

Total execution time: 00:00:00.130

In [8]:
CREATE EXTENSION postgis;



: extension "postgis" already exists

Total execution time: 00:00:00.008

In [44]:
CREATE TABLE buildings(id SERIAL PRIMARY KEY, geometry geometry, name VARCHAR);
CREATE TABLE roads(id SERIAL PRIMARY KEY, geometry geometry, name VARCHAR);
CREATE TABLE poi(id SERIAL PRIMARY KEY, geometry geometry, name VARCHAR);

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.046

In [45]:
INSERT INTO roads (geometry, name) VALUES
    ('LINESTRING(7.5 10.5, 7.5 0)', 'roadY'),
    ('LINESTRING(0 4.5, 12 4.5)', 'roadX');

INSERT INTO buildings (geometry, name) VALUES
    ('POLYGON((3 6, 5 6, 5 8, 3 8, 3 6))', 'BuildingC'),
    ('POLYGON((4 5, 6 5, 6 7, 4 7, 4 5))', 'BuildingB'),
    ('POLYGON((9 8, 10 8, 10 9, 9 9, 9 8))', 'BuildingD'),
    ('POLYGON((8 1.5, 10.5 1.5, 10.5 4 , 8 4, 8 1.5))', 'BuildingA'),
    ('POLYGON((1 1, 2 1, 2 2, 1 2, 1 1))', 'BuildingF');

INSERT INTO poi (geometry, name)  VALUES
    ('POINT(1 3.5)', 'G'),
    ('POINT(5.5 1.5)', 'H'),
    ('POINT(6.5 6)', 'J'),
    ('POINT(6 9.5)', 'K'),
    ('POINT(9.5 6)', 'I');



Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.017

In [46]:
select * from roads;


(2 row(s) affected)

Total execution time: 00:00:00.021

id,geometry,name
1,0102000000020000000000000000001E4000000000000025400000000000001E400000000000000000,roadY
2,0102000000020000000000000000000000000000000000124000000000000028400000000000001240,roadX


: relation "building" does not exist

In [48]:
select * from buildings;


(5 row(s) affected)

Total execution time: 00:00:00.006

id,geometry,name
1,010300000001000000050000000000000000000840000000000000184000000000000014400000000000001840000000000000144000000000000020400000000000000840000000000000204000000000000008400000000000001840,BuildingC
2,01030000000100000005000000000000000000104000000000000014400000000000001840000000000000144000000000000018400000000000001C4000000000000010400000000000001C4000000000000010400000000000001440,BuildingB
3,010300000001000000050000000000000000002240000000000000204000000000000024400000000000002040000000000000244000000000000022400000000000002240000000000000224000000000000022400000000000002040,BuildingD
4,010300000001000000050000000000000000002040000000000000F83F0000000000002540000000000000F83F00000000000025400000000000001040000000000000204000000000000010400000000000002040000000000000F83F,BuildingA
5,01030000000100000005000000000000000000F03F000000000000F03F0000000000000040000000000000F03F00000000000000400000000000000040000000000000F03F0000000000000040000000000000F03F000000000000F03F,BuildingF


In [49]:
select * from poi;

(5 row(s) affected)

Total execution time: 00:00:00.026

id,geometry,name
1,0101000000000000000000F03F0000000000000C40,G
2,01010000000000000000001640000000000000F83F,H
3,01010000000000000000001A400000000000001840,J
4,010100000000000000000018400000000000002340,K
5,010100000000000000000023400000000000001840,I


a. Wyznacz całkowitą długość dróg w analizowanym mieście.

In [50]:
select sum(ST_LENGTH(geometry)) from roads; 

(1 row(s) affected)

Total execution time: 00:00:00.034

sum
22.5


b. Wypisz geometrię (WKT), pole powierzchni oraz obwód poligonu reprezentującego budynek o nazwie BuildingA.

In [58]:
select ST_AsText(geometry) as WKT,
       ST_AREA(geometry) as area,
       ST_PERIMETER(geometry) as perimeter
       from buildings where name = 'BuildingA';

(1 row(s) affected)

Total execution time: 00:00:00.021

wkt,area,perimeter
"POLYGON((8 1.5,10.5 1.5,10.5 4,8 4,8 1.5))",6.25,10.0


c. Wypisz nazwy i pola powierzchni wszystkich poligonów w warstwie budynki. Wyniki

posortuj alfabetycznie.

In [60]:
select  name, ST_AREA(geometry) as area
       from buildings order by name
            

(5 row(s) affected)

Total execution time: 00:00:00.031

name,area
BuildingA,6.25
BuildingB,4.0
BuildingC,4.0
BuildingD,1.0
BuildingF,1.0


d. Wypisz nazwy i obwody 2 budynków o największej powierzchni.

In [69]:
SELECT name, ST_PERIMETER(geometry) as perimeter  
    FROM ( 
        SELECT name, geometry, ST_AREA(geometry) as area
        FROM buildings
        ORDER BY area DESC
        LIMIT(2)
        ) as b

(2 row(s) affected)

Total execution time: 00:00:00.023

name,perimeter
BuildingA,10.0
BuildingB,8.0


e. Wyznacz najkrótszą odległość między budynkiem BuildingC a punktem K.

In [81]:
SELECT ST_DISTANCE(b.geometry, p.geometry) 
        FROM buildings as b, poi as p
        WHERE b.name = 'BuildingC' AND p.name = 'K'


(1 row(s) affected)

Total execution time: 00:00:00.018

st_distance
1.8027756377319946


f. Wypisz pole powierzchni tej części budynku BuildingC, która znajduje się w odległości większej niż 0.5 od budynku BuildingB

In [25]:
select ST_Area(ST_Difference(c.geometry, ST_Buffer(b.geometry, 0.5)))
        from 
        (select geometry from buildings where name = 'BuildingB') as b,
        (select geometry from buildings where name = 'BuildingC') as c;


(1 row(s) affected)

Total execution time: 00:00:00.023

st_area
1.804909677983872


g. Wybierz te budynki, których centroid (ST\_Centroid) znajduje się powyżej drogi o nazwie RoadX. 8. Oblicz pole powierzchni tych części budynku BuildingC i poligonu o współrzędnych (4 7, 6 7, 6 8, 4 8, 4 7), które nie są wspólne dla tych dwóch obiektów.

In [24]:
with up as (select ST_Y(ST_CENTROID(geometry)) as y from roads where name = 'roadX')

select name from buildings as b, up where  ST_Y(ST_CENTROID(b.geometry)) > up.y;

(3 row(s) affected)

Total execution time: 00:00:00.023

name
BuildingC
BuildingB
BuildingD


Oblicz pole powierzchni tych części budynku BuildingC i poligonu o współrzędnych (4 7, 6 7, 6 8, 4 8, 4 7), które nie są wspólne dla tych dwóch obiektów.

In [21]:
select ST_Area(ST_SymDifference(geometry, ST_GeomFromText('POLYGON((4 7, 6 7, 6 8, 4 8, 4 7))')))
     from buildings
     where name = 'BuildingC';

(1 row(s) affected)

Total execution time: 00:00:00.023

st_area
4.0


CLEAR

In [43]:
drop table poi;
drop table roads;
drop table buildings;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.036